In [ ]:
import datetime
import time
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [ ]:
file_path=input("Enter the path to your csv file: ")

file_path=str(file_path)
#/Users/thapliyaa/Desktop/CS_IA/DOM VA Power Detailed Energy Usage.csv
cost_KWH=input("How much do you pay per KWH? Insert your number as a decimal. ")
cost_KWH=float(cost_KWH)
total_system_cost=input("How much did you pay for your solar panels? Insert your number as a whole number with no commas. ")
total_system_cost=int(total_system_cost)

In [ ]:
#Reads the usage data csv file and turns it into a dataframe
energydata=pd.read_csv(file_path, dtype='unicode')
#Deletes unnecessary columns of data
energydata.drop(['Account No', 'Recorder ID', 'Day', 'Month', 'Year','New Date'], axis=1, inplace=True)
#Deletes unnecessary rows of data
energydata.drop(index=energydata.iloc[0:21].index.tolist(), inplace=True)
#Changes the date column to a datetime format
energydata['DATE']=pd.to_datetime(energydata['Date'].values)
del energydata['Date']
energydata.rename(columns={'TOTAL Consumption':'KWH Consumption', 'TOTALProduction':'KWH Production'}, inplace=True)
#Converts data type from float64 to float32 to reduce memory
energydata['KWH Consumption']=energydata['KWH Consumption'].astype(np.float32)
energydata['KWH Production']=energydata['KWH Production'].astype(np.float32)

In [ ]:
#Prints the format of dataframe: columns names and first five rows
energydata.head()

In [ ]:
#Prints general statistical information about data
energydata.describe()

In [ ]:
#Prints data type and data size information
energydata.info()

In [ ]:
#Splits consumption data into separate dataframe for prediction

consumptionY=energydata['KWH Consumption']
consumptionY.index=energydata['DATE']
consumptionY.head()
#Features that can be predicted
consumptionX = energydata[['KWH Consumption', 'KWH Production']]
#Target Variable: what is getting predicted
consumptionY = energydata['KWH Consumption']
#Tuple unpacking to grab train and test data sets
#Test size is the % of data set that is allocated to the test set
X_train, X_test, y_train, y_test = train_test_split(consumptionX, consumptionY, test_size=0.7)
#Creates Linear Regression object
consumption_linearmodel = LinearRegression()
#Fits linear model
consumption_linearmodel.fit(X_train, y_train)
#Array of predictions of KWH Consumption
consumption_predictions=consumption_linearmodel.predict(X_test)
#Added predicted consumption data to a separate csv file
consumptiondata=pd.read_csv("/Users/thapliyaa/Desktop/CS_IA/PredictedConsumption.csv", dtype='unicode')
consumptiondata['Prediction']=consumption_predictions.tolist()
consumptiondata['DATE']=pd.to_datetime(consumptiondata['11/21/20'].values)
del consumptiondata['11/21/20']

In [ ]:
data=input("Which data would you like to see? (production or consumption)")
data=str(data)
graph=input("What kind of graph would you like to see? (line, scatter, or bar) ")
graph=str(graph)
ans=input("Would you like to specify with time intervals? (yes or no)")
ans=str(ans)
if data=="production":
    if graph=="line":
        if ans=="yes":
            start=input("What day would you like to start on? (YYYY-MM-DD)")

            start=str(start)
            end=input("What day would you like to end on? (YYYY-MM-DD)")

            end=str(end)
            productionY.loc[start:end].plot(figsize=(20,5))
    else:
        productionY.plot(figsize=(20,5))
    if graph=="scatter":
        plt.scatter(energydata['DATE'], productionY)
    if graph=="bar":
        plt.bar(energydata['DATE'], productionY)
if data=="consumption":
    if graph=="line":
        if ans=="yes":
            start=input("What day would you like to start on? (YYYY-MM-DD) ")
            start=str(start)
            end=input("What day would you like to end on? (YYYY-MM-DD) ")
            end=str(end)
            consumptionY.loc[start:end].plot(figsize=(20,5))
    else:
        consumptionY.plot(figsize=(20,5))
    if graph=="scatter":
        plt.scatter(energydata['DATE'], consumptionY)
    if graph=="bar":
        plt.bar(energydata['DATE'], consumptionY)

In [ ]:
y1=productiondata['Prediction']
y1.index=productiondata['DATE']
p_graph1=y1.plot(figsize=(30,5), color='Red', alpha=0.6, fontsize=20)
p_graph1.set_ylabel('Predicted KWH Consumption')
p_graph2=productionY.plot(figsize=(30,5), title='Past and Future KWH Production', grid=True, fontsize=20, alpha=0.6, color='Blue')
p_graph2.set_xlabel('DATE', fontsize=20)
p_graph2.set_ylabel('KWH Production', fontsize=20)
p_graph2.title.set_size(20)
plt.tight_layout()
p_graph2.legend(['Predicted KWH Production','KWH Production'], fontsize=20, loc='upper right')

In [ ]:
y2=consumptiondata['Prediction']
y2.index=consumptiondata['DATE']
c_graph1=y2.plot(figsize=(30,5), color='Red', alpha=0.6, fontsize=20)
c_graph1.set_ylabel('Predicted KWH Consumption')
c_graph2=consumptionY.plot(figsize=(30,5), title='Past and Future KWH Consumption', grid=True, fontsize=20, alpha=0.6, color='Blue')
c_graph2.set_xlabel('DATE', fontsize=20)
c_graph2.set_ylabel('KWH Consumption', fontsize=20)
c_graph2.title.set_size(20)
plt.tight_layout()
c_graph2.legend(['Predicted KWH Consumption','KWH Consumption'], fontsize=20, loc='upper right')

In [ ]:
cumulative_df=pd.DataFrame(energydata, columns=['DATE'])
consumption_col=energydata.loc[:,'KWH Consumption']
consumption_arr=consumption_col.values
consumption_arr=consumption_arr.astype(int)
production_col=energydata.loc[:, 'KWH Production']
production_arr=production_col.values
for i in range(1,1757):
    production_arr[i]=(production_arr[i-1]+production_arr[i])
for i in range(1,1757):
    consumption_arr[i]=(consumption_arr[i-1]+consumption_arr[i])

In [ ]:
cumulative_df['Cumulative Consumption']=consumption_arr.tolist()
cumulative_df['Cumulative Production']=production_arr.tolist()

In [ ]:
totalY=cumulative_df['Cumulative Consumption']

totalY.index=cumulative_df['DATE']
totalY.head()
y3=cumulative_df['Cumulative Production']
y3.index=cumulative_df['DATE']
t_graph1=y3.plot(figsize=(20,10), color='Red', alpha=0.6, fontsize=20, linewidth=5)
t_graph2=totalY.plot(figsize=(20,10), title='Cumulative KWH Production and Consumption',grid=True, fontsize=20, alpha=0.6, color='Blue', linewidth=5)
t_graph2.set_xlabel('DATE', fontsize=20)
t_graph2.set_ylabel('Cumulative KWH', fontsize=20)
t_graph2.title.set_size(20)
plt.tight_layout()
t_graph2.legend(['Cumulative KWH Production','Cumulative KWH Consumption'], fontsize=20, loc='upper left')

In [ ]:
predictions_df=pd.DataFrame(consumptiondata, columns=['DATE'])

pred_consumption=consumptiondata.loc[:,'Prediction']
pred_consumption_arr=pred_consumption.values
pred_production=productiondata.loc[:,'Prediction']
pred_production_arr=pred_production.values
pred_consumption_arr[0]=consumption_arr[1756]
pred_production_arr[0]=production_arr[1756]
#predictions_df
for i in range(1,1230):
    pred_production_arr[i]=(pred_production_arr[i-1]+pred_production_arr[i])
    #print(production_arr[i])
for i in range(1,1230):
    pred_consumption_arr[i]=(pred_consumption_arr[i-1]+pred_consumption_arr[i])
    #consumption_arr[i]=consumption_arr[i]*cost_KWH
    #print(consumption_arr[i])
predictions_df['Predicted Consumption']=pred_consumption_arr.tolist()
predictions_df['Predicted Production']=pred_production_arr.tolist()

In [ ]:
totalY=cumulative_df['Cumulative Consumption']

totalY.index=cumulative_df['DATE']
t_graph2=totalY.plot(figsize=(25,10), title='Cumulative KWH Production and Consumption',
grid=True, fontsize=20, alpha=0.6, color='Blue', linewidth=5)
#y3=cumulative_df['Cumulative Production']
#y3.index=cumulative_df['DATE']
#t_graph1=y3.plot(figsize=(20,10), color='Red', alpha=0.6, fontsize=20,

#t_graph2.set_xlabel('DATE', fontsize=20)
#t_graph2.set_ylabel('Cumulative KWH', fontsize=20)
#t_graph2.title.set_size(20)
#plt.tight_layout()
predY=predictions_df['Predicted Consumption']
predY.index=predictions_df['DATE']
t_graph3=predY.plot(figsize=(25,10), title='Predicted KWH Production andConsumption',grid=True, fontsize=20, alpha=0.6, color='Green', linewidth=5)
y4=predictions_df['Predicted Production']
y4.index=predictions_df['DATE']
#t_graph4=y4.plot(figsize=(20,15), grid=True, fontsize=20, alpha=0.6, color='Orange', linewidth=5)
t_graph3.legend(['KWH Production','Predicted KWH Production'], fontsize=20, loc='upper left')

In [ ]:
saved=int(production_arr[1756]*cost_KWH)

print("Total saved from 1/27/2016 to 11/20/2020: $"+str(saved))
pred_saved=int((pred_production_arr[1229]-production_arr[1756])*cost_KWH)
print("Total predicted to save from 11/21/2020 to 4/24/2024: $"+str(pred_saved))
total_saved=int(saved+pred_saved)
print("Total savings in 7.4 years: $"+str(total_saved))
total_system_cost=16723
payback_period=float(round((total_system_cost/total_saved), 2)*7.4)
print("Client will break even in "+str(payback_period)+" years")